In [41]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from statsmodels.sandbox.regression import gmm
import shutil
from pathlib import Path
os.getcwd()

'/Users/rubenexojo/Library/Mobile Documents/com~apple~CloudDocs/phd/uni/courses/finance/finance-3/hw/code'

In [46]:
def find_file(name):
    return "data/" + name

def df_essential(df):
    drop_cols=["LeadReal", "MidReal", "LagReal"]
    for col in drop_cols:
        df = df.drop(col, 1)
    return df

def df_include_constant_columns(df):
    df["const"] = [1]*len(df)
    cols = df.columns.tolist()
    cols = [cols[0]] + ["const"] + cols[1:-1]
    return df[cols]

def fix_date(date):
    year    = [date[i] for i in range(4)]
    month   = [date[i] for i in range(4,6)]
    day     = [date[i] for i in range(6,8)]
    year    = ''.join(year)
    month   = ''.join(month)
    day     = ''.join(day)
    date = [day, month, year]
    date = '/'.join(date)
    return date

def df_fix_date(df):
    date_col = df['dt']
    for i in range(len(df)):
        date_col[i] = fix_date(str(date_col[i]))
    df['dt'] = date_col
    return df
    
def df_annualise(df):
    cols=["LeadR", "MidR", "LagR", "Lead", "Mid", "Lag", "LL", "LLStrong"]
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')*12*100
    return df

def df_excess_returns(df,  annualised = True):
    all_cols =  df.columns.tolist()
    rf = pd.to_numeric(df["rf"], errors='coerce')*12
    cols=["Lead", "Mid", "Lag"]
    move = 1
    for col in cols:
        new_col=f"ex_{col}"
        if annualised:
            loc = all_cols.index(col) + move
            df.insert(loc, new_col, pd.to_numeric(df[col], errors='coerce') - rf)
        else:
            loc = all_cols.index(col) + move
            df.insert(loc, new_col, pd.to_numeric(df[col], errors='coerce')*12*100 - rf)
        move += 1
    cols=["LL", "LLStrong"]
    for col in cols:
        new_col=f"ex_{col}"
        if annualised:
            loc = all_cols.index(col) + move
            df.insert(loc, new_col, pd.to_numeric(df[col], errors='coerce'))
        else:
            df.insert(loc, new_col, pd.to_numeric(df[col], errors='coerce'))*12*100
        move += 1
    return df

def df_merge(df1, df2):
    df = df1.merge(df2, how='inner', on="dtt", left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_left', '_right'), copy=True, indicator=True, validate="one_to_one")
    df.drop('dtt', inplace=True, axis=1)
    df.drop('_merge', inplace=True, axis=1)
    return df

def take_factor_sheet(): 
    master_file = find_file("Data_master.xlsx")
    rf_file     = find_file("FF4_monthlyKF.csv")
    # read xlsx file
    df     = pd.read_excel(master_file, sheet_name="T1_ptfs", names=["dt", "LeadR", "MidR", "LagR", "Lead", "Mid", "Lag", "LL", "LLStrong", "mktrf", "smb", "hml"], dtype=str)
    df_rf  = pd.read_csv(rf_file, sep=";", names=["dtt", "v2", "v3", "v4", "rf"], dtype=str)
    df_rf  = df_rf[["dtt", "rf"]]
    df2    = df_rf.copy()
    # df = df_essential(df)
    # include a column of ones to calculate the average return
    df = df_include_constant_columns(df)
    # annualise data (we have it daily)
    df = df_annualise(df)
    # merge
    df["dtt"]=df["dt"].str.slice(0, 6)
    df1= df.copy()
    df = df_merge(df1, df2)
    # include columns of excess returns
    df = df_excess_returns(df)
    df.to_excel('csv/mine.xlsx')
    return df

def take_factors(df):
    return df.keys().tolist()

def factors_order(factors):
    flags   = [k for k in range(len(factors))]
    zip_f   = {}
    for flag in flags:
        zip_f[factors[flag]] = flag
    return zip_f

def LL_portfolio_sorting(F, zip_f, excess = False):
    if excess:
        return [F[zip_f['ex_Lead']], F[zip_f['ex_Mid']], F[zip_f['ex_Lag']], F[zip_f['LL']], F[zip_f['LLStrong']]]
    else:
        return [F[zip_f['LeadR']], F[zip_f['MidR']], F[zip_f['LagR']], F[zip_f['LL']], F[zip_f['LLStrong']]]

def tex_file(table, title):
    return "tex/" + table + "_" + title + ".txt"
    
def create_txt(table, title, text):
    file    = Path(tex_file(table, title))
    file.write_text(f"{text}\n\n")

def reg(Y, X, lags, table, title, df):
    regs=[None]*len(Y)
    for i, y in enumerate(Y):
        y   = df[y]
        reg = sm.OLS(y.astype(float), X.astype(float)).fit(cov_type='HAC', cov_kwds={'maxlags':lags})
        regs[i]=reg  
    new_Y=[y.replace("ex_", "") for y in Y]

    sum = summary_col(results=regs, float_format='%0.2f', model_names=new_Y, stars=True, info_dict=None,  drop_omitted=True)
    text    = sum.as_latex()
    create_txt(table, title, text)

In [49]:
df      = take_factor_sheet('T1_ptfs')
F       = take_factors(df)
zip_f   = factors_order(F)
print(zip_f)
lags    = 24
table   = "T1"
# AVERAGE RETURN 
Y       = LL_portfolio_sorting(F, zip_f)
row     = "average_return"
X       = df["const"]
reg(Y, X, lags, table, row, df)
# # CAPM 
Y       = LL_portfolio_sorting(F, zip_f, True)
row     = "capm"
X       = df[["const", "mktrf"]]
reg(Y, X, lags, table, row, df)
# # FFM
# Y       = LL_portfolio_sorting(F, zip_f, True)
row     = "ffm"
X       = df[["const", "mktrf", "smb", "hml"]]
reg(Y, X, lags, table, row, df)

{'dt': 0, 'const': 1, 'LeadR': 2, 'MidR': 3, 'LagR': 4, 'Lead': 5, 'ex_Lead': 6, 'Mid': 7, 'ex_Mid': 8, 'Lag': 9, 'ex_Lag': 10, 'LL': 11, 'ex_LL': 12, 'LLStrong': 13, 'ex_LLStrong': 14, 'mktrf': 15, 'smb': 16, 'hml': 17, 'rf': 18}


In [576]:
df

,dt,const,LeadR,MidR,LagR,Lead,ex_Lead,Mid,ex_Mid,Lag,ex_Lag,LL,ex_LL,LLStrong,ex_LLStrong,mktrf,smb,hml,rf
0,19720131,1,0.02588464305351764,0.027900294457035336,0.008135043631581408,33.981280,30.501280,36.400061,32.920061,12.681760,9.201760,21.299519,21.299519,-12.363476,-12.363476,2.49,6.11,2.03,0.29
1,19720229,1,0.054071645659457684,0.017870025488794065,0.036488204419470864,70.711218,67.711218,27.269273,24.269273,49.611088,46.611088,21.100129,21.100129,7.417432,7.417432,2.87,1.38,-2.76,0.25
2,19720330,1,0.02380823799631148,-0.0014409777942145565,0.02828417856140748,28.569886,25.329886,-1.729173,-4.969173,33.941014,30.701014,-5.371129,-5.371129,-14.899818,-14.899818,0.63,-0.27,-1.69,0.27
3,19720428,1,0.012421677713027577,0.004786537059166818,-0.010640747832879036,17.804564,14.324564,8.642395,5.162395,-9.870347,-13.350347,27.674911,27.674911,21.298575,21.298575,0.29,0,0.22,0.29
4,19720531,1,0.021549861312477755,0.017493138029221932,-0.0013931916678239168,28.751400,25.151400,23.883332,20.283332,1.219736,-2.380264,27.531664,27.531664,13.469281,13.469281,1.25,-2.79,-2.69,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,20120831,1,0.025419844987527668,0.018989330985077456,0.025086635533552074,36.656477,36.536477,28.939860,28.819860,36.256625,36.136625,0.399851,0.399851,-20.295849,-20.295849,2.55,0.71,0.55,0.01
488,20120928,1,0.027982022538467266,0.022259680351975265,0.015075429436017862,39.496184,39.376184,32.629373,32.509373,24.008272,23.888272,15.487912,15.487912,34.295464,34.295464,2.73,0.49,1.57,0.01
489,20121031,1,-0.036006757840210235,-0.005938089804156147,-0.049763952314544606,-40.268944,-40.388944,-4.186542,-4.306542,-56.777577,-56.897577,16.508633,16.508633,39.900464,39.900464,-1.76,-1.07,4.14,0.01
490,20121130,1,0.015113435826048191,0.007416783071997779,0.0018261453466293582,15.742878,15.622878,6.506895,6.386895,-0.201871,-0.321871,15.944749,15.944749,0.715445,0.715445,0.78,0.67,-1.1,0.01
